<a href="https://colab.research.google.com/github/dhrumilrdg142/Next-word-pridictar/blob/main/Copy_of_Movies_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
tmdb_tmdb_movie_metadata_path = kagglehub.dataset_download('tmdb/tmdb-movie-metadata')

print('Data source import complete.')


Using Colab cache for faster access to the 'tmdb-movie-metadata' dataset.
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [ ]:
movies= pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies= movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
'''
genres
id
keywords
title
overview
release date
cast
crew
'''

movies=movies[['movie_id','title','genres','keywords','cast','crew','overview']]

In [ ]:
movies

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.info()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

def convert(obj):
    L= []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
import ast

def convert3(obj):
    L= []
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
import ast

def fetch_Director(obj):
    L= []
    counter=0
    for i in ast.literal_eval(obj):
        if i['job'] =='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_Director)

In [ ]:
movies.head()

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords']+ movies['cast']+movies['crew']

In [ ]:
new_df= movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
new_df['tags']= new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
cv.get_feature_names_out()

In [ ]:
#donot use euclidean distance
#instead use cosine distance

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity[1]

In [ ]:
def recommend(movie):
    movie_index= new_df[new_df['title']==movie].index[0]
    distances= similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Batman Begins')

In [ ]:
 import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))